In [100]:
import os
import toml

In [101]:
# Load the TOML file
with open('config.toml', 'r') as f:
    config = toml.load(f)

# Set the environment variables
for key, value in config.items():
    os.environ[key] = str(value)


In [102]:
import ibis
con = ibis.duckdb.connect('md:Eunomia')

In [103]:
print(con.list_tables())

['CARE_SITE', 'CDM_SOURCE', 'COHORT', 'COHORT_ATTRIBUTE', 'CONCEPT', 'CONCEPT_ANCESTOR', 'CONCEPT_CLASS', 'CONCEPT_RELATIONSHIP', 'CONCEPT_SYNONYM', 'CONDITION_ERA', 'CONDITION_OCCURRENCE', 'COST', 'DEATH', 'DEVICE_EXPOSURE', 'DOMAIN', 'DOSE_ERA', 'DRUG_ERA', 'DRUG_EXPOSURE', 'DRUG_STRENGTH', 'FACT_RELATIONSHIP', 'LOCATION', 'MEASUREMENT', 'METADATA', 'NOTE', 'NOTE_NLP', 'OBSERVATION', 'OBSERVATION_PERIOD', 'PAYER_PLAN_PERIOD', 'PERSON', 'PROCEDURE_OCCURRENCE', 'PROVIDER', 'RELATIONSHIP', 'SOURCE_TO_CONCEPT_MAP', 'SPECIMEN', 'VISIT_DETAIL', 'VISIT_OCCURRENCE', 'VOCABULARY', 'ambient_air_quality', 'coxibVsNonselVsGiBleed', 'hacker_news', 'outcomes', 'rideshare', 'service_requests', 'taxi']


In [104]:
import re
from pydantic import BaseModel, confloat
from datetime import datetime, date
import ibis

con = ibis.duckdb.connect('md:Eunomia')

table_name = 'FACT_RELATIONSHIP'

table = con.table(f'{table_name}')

def ibis_schema_to_pydantic(schema_str: str, class_name: str) -> str:
    # Parse the schema string into a dictionary
    schema_dict = {}
    for match in re.findall(r'(\w+)\s+(\w+)', schema_str):
        schema_dict[match[0]] = match[1]

    # Generate the Pydantic fields
    pydantic_fields = []
    for name, dtype in schema_dict.items():
        if name.endswith('_DATETIME'):
            pydantic_type = 'datetime'
            default_value = 'None'
        elif name.endswith('_DATE'):
            pydantic_type = 'datetime'
            default_value = 'None'
        elif name.endswith('_ID'):
            pydantic_type = 'int'
            default_value = 'None'
        else:
            pydantic_type = 'int' if dtype in ('int32', 'int64', 'float64') else 'str'
            default_value = 'None' if pydantic_type == 'Optional[str]' else '...'
        
        # Handle 'None' string as default value for datetime fields
        if pydantic_type == 'datetime' and default_value == 'None':
            default_value = None
        
        pydantic_fields.append(f'{name.lower()}: Optional[{pydantic_type}] = Field({default_value}, alias=\'{name}\')')

    # Generate the Pydantic model with the derived class name
    class_name = ''.join(word.capitalize() for word in re.split(r'[^a-zA-Z0-9_]', class_name))
    class_name = class_name[:1].upper() + class_name[1:]
    class_name = re.sub(r'_(\w)', lambda m: m.group(1).upper(), class_name)
    return f'class {class_name}(BaseModel):\n    ' + '\n    '.join(pydantic_fields)

schema_str = str(table.schema())
# Derive the class name from the table name
class_name = ''.join(word.capitalize() for word in re.split(r'[^a-zA-Z0-9_]', table_name))
# Test the function
print(ibis_schema_to_pydantic(schema_str, class_name))

class FactRelationship(BaseModel):
    domain_concept_id_1: Optional[int] = Field(..., alias='DOMAIN_CONCEPT_ID_1')
    fact_id_1: Optional[int] = Field(..., alias='FACT_ID_1')
    domain_concept_id_2: Optional[int] = Field(..., alias='DOMAIN_CONCEPT_ID_2')
    fact_id_2: Optional[int] = Field(..., alias='FACT_ID_2')
    relationship_concept_id: Optional[int] = Field(None, alias='RELATIONSHIP_CONCEPT_ID')


In [105]:
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, Type
from datetime import date, datetime
import pandas as pd
import numpy as np

def validate_dataframe(df: pd.DataFrame, model: Type[BaseModel]) -> pd.DataFrame:
    for index, row in df.iterrows():
        try:
            # Convert values to None if they are NaN or None
            row_dict = {k: v if not pd.isna(v) and v is not None else None for k, v in row.to_dict().items()}
            model(**row_dict)
        except ValidationError as e:
            print(f"Validation error for row {index}: {e}")
    return df

class FactRelationship(BaseModel):
    domain_concept_id_1: Optional[int] = Field(..., alias='DOMAIN_CONCEPT_ID_1')
    fact_id_1: Optional[int] = Field(..., alias='FACT_ID_1')
    domain_concept_id_2: Optional[int] = Field(..., alias='DOMAIN_CONCEPT_ID_2')
    fact_id_2: Optional[int] = Field(..., alias='FACT_ID_2')
    relationship_concept_id: Optional[int] = Field(None, alias='RELATIONSHIP_CONCEPT_ID')

In [106]:
df = table.execute()

validate_dataframe(df, FactRelationship)

,DOMAIN_CONCEPT_ID_1,FACT_ID_1,DOMAIN_CONCEPT_ID_2,FACT_ID_2,RELATIONSHIP_CONCEPT_ID
